In [1]:
%load_ext autoreload
%autoreload 2
from typing import *

In [3]:
from utils import load_input

# Part 1 

As your flight slowly drifts through the sky, the Elves at the Mythical Information Bureau at the North Pole contact you. They'd like some help debugging a malfunctioning experimental energy source aboard one of their super-secret imaging satellites.

The experimental energy source is based on cutting-edge technology: a set of Conway Cubes contained in a pocket dimension! When you hear it's having problems, you can't help but agree to take a look.

The pocket dimension contains an infinite 3-dimensional grid. At every integer 3-dimensional coordinate (x,y,z), there exists a single cube which is either active or inactive.

In the initial state of the pocket dimension, almost all cubes start inactive. The only exception to this is a small flat region of cubes (your puzzle input); the cubes in this region start in the specified active (#) or inactive (.) state.

The energy source then proceeds to boot up by executing six cycles.

Each cube only ever considers its neighbors: any of the 26 other cubes where any of their coordinates differ by at most 1. For example, given the cube at x=1,y=2,z=3, its neighbors include the cube at x=2,y=2,z=2, the cube at x=0,y=2,z=3, and so on.

During a cycle, all cubes simultaneously change their state according to the following rules:

If a cube is active and exactly 2 or 3 of its neighbors are also active, the cube remains active. Otherwise, the cube becomes inactive.
If a cube is inactive but exactly 3 of its neighbors are active, the cube becomes active. Otherwise, the cube remains inactive.
The engineers responsible for this experimental energy source would like you to simulate the pocket dimension and determine what the configuration of cubes should be at the end of the six-cycle boot process.

For example, consider the following initial state:
```
.#.
..#
###
```
Even though the pocket dimension is 3-dimensional, this initial state represents a small 2-dimensional slice of it. (In particular, this initial state defines a 3x3x1 region of the 3-dimensional space.)

Simulating a few cycles from this initial state produces the following configurations, where the result of each cycle is shown layer-by-layer at each given z coordinate (and the frame of view follows the active cells in each cycle):

Before any cycles:
```
z=0
.#.
..#
###


After 1 cycle:

z=-1
#..
..#
.#.

z=0
#.#
.##
.#.

z=1
#..
..#
.#.


After 2 cycles:

z=-2
.....
.....
..#..
.....
.....

z=-1
..#..
.#..#
....#
.#...
.....

z=0
##...
##...
#....
....#
.###.

z=1
..#..
.#..#
....#
.#...
.....

z=2
.....
.....
..#..
.....
.....


After 3 cycles:

z=-2
.......
.......
..##...
..###..
.......
.......
.......

z=-1
..#....
...#...
#......
.....##
.#...#.
..#.#..
...#...

z=0
...#...
.......
#......
.......
.....##
.##.#..
...#...

z=1
..#....
...#...
#......
.....##
.#...#.
..#.#..
...#...

z=2
.......
.......
..##...
..###..
.......
.......
.......
```
After the full six-cycle boot process completes, 112 cubes are left in the active state.

Starting with your given initial configuration, simulate six cycles. How many cubes are left in the active state after the sixth cycle?




In [12]:
test = """.#.
..#
###
"""
data = load_input(17, splitlines=False)

In [103]:
from __future__ import annotations
from typing import NamedTuple
from itertools import product, combinations

class Cube(NamedTuple):
    x: int = 0
    y: int = 0 
    z: int = 0
    
    def __add__(self, other: Cube) -> Cube:
        return Cube(
            self.x + other.x,
            self.y + other.y,
            self.z + other.z,
        )
        
    @property
    def neighbors(self) -> Set[Cube]:
        offsets = (Cube(*xyz) for xyz in product([-1, 0, 1], repeat=3) if xyz != (0, 0, 0))
        return set(self + other for other in offsets)
    

#Coord = namedtuple("Coord", "x y z")

In [116]:
from collections import Counter

class Game:
    def __init__(self):
        self.state = set()
    
    @classmethod
    def from_str(cls, s: str, cube_cls) -> Game:
        rows = s.splitlines()
        init_state = set(cube_cls(x, y) for y, row in enumerate(rows) for x, v in enumerate(row) if v=="#")
        c = cls()
        c.state = init_state
        return c
    
    def step(self):
        new_state = set()
        nc = Counter()

        for cube in self.state:
            neighbors = cube.neighbors
            if sum(neighbor in self.state for neighbor in neighbors) in (2, 3):
                new_state.add(cube)
            nc.update(neighbors - self.state)
        
        new_state.update(k for k,v in nc.items() if v==3)
        
        self.state = new_state
    
    def step_n(self, n: int):
        print(f"Initial active: {self.n_active}")
        for i in range(n):
            self.step()
            print(f" After {i+1} cycles: {self.n_active}")
        
    @property
    def n_active(self):
        return len(self.state)        

In [117]:
g = Game.from_str(data, Cube)
g.n_active

28

In [119]:
g.step_n(6)

Initial active: 28
 After 1 cycles: 72
 After 2 cycles: 98
 After 3 cycles: 143
 After 4 cycles: 198
 After 5 cycles: 289
 After 6 cycles: 384


# Part 2

For some reason, your simulated results don't match what the experimental energy source engineers expected. Apparently, the pocket dimension actually has four spatial dimensions, not three.

The pocket dimension contains an infinite 4-dimensional grid. At every integer 4-dimensional coordinate (x,y,z,w), there exists a single cube (really, a hypercube) which is still either active or inactive.

Each cube only ever considers its neighbors: any of the 80 other cubes where any of their coordinates differ by at most 1. For example, given the cube at x=1,y=2,z=3,w=4, its neighbors include the cube at x=2,y=2,z=3,w=3, the cube at x=0,y=2,z=3,w=4, and so on.

The initial state of the pocket dimension still consists of a small flat region of cubes. Furthermore, the same rules for cycle updating still apply: during each cycle, consider the number of active neighbors of each cube.

For example, consider the same initial state as in the example above. Even though the pocket dimension is 4-dimensional, this initial state represents a small 2-dimensional slice of it. (In particular, this initial state defines a 3x3x1x1 region of the 4-dimensional space.)

Simulating a few cycles from this initial state produces the following configurations, where the result of each cycle is shown layer-by-layer at each given z and w coordinate:

Before any cycles:
```
z=0, w=0
.#.
..#
###


After 1 cycle:

z=-1, w=-1
#..
..#
.#.

z=0, w=-1
#..
..#
.#.

z=1, w=-1
#..
..#
.#.

z=-1, w=0
#..
..#
.#.

z=0, w=0
#.#
.##
.#.

z=1, w=0
#..
..#
.#.

z=-1, w=1
#..
..#
.#.

z=0, w=1
#..
..#
.#.

z=1, w=1
#..
..#
.#.


After 2 cycles:

z=-2, w=-2
.....
.....
..#..
.....
.....

z=-1, w=-2
.....
.....
.....
.....
.....

z=0, w=-2
###..
##.##
#...#
.#..#
.###.

z=1, w=-2
.....
.....
.....
.....
.....

z=2, w=-2
.....
.....
..#..
.....
.....

z=-2, w=-1
.....
.....
.....
.....
.....

z=-1, w=-1
.....
.....
.....
.....
.....

z=0, w=-1
.....
.....
.....
.....
.....

z=1, w=-1
.....
.....
.....
.....
.....

z=2, w=-1
.....
.....
.....
.....
.....

z=-2, w=0
###..
##.##
#...#
.#..#
.###.

z=-1, w=0
.....
.....
.....
.....
.....

z=0, w=0
.....
.....
.....
.....
.....

z=1, w=0
.....
.....
.....
.....
.....

z=2, w=0
###..
##.##
#...#
.#..#
.###.

z=-2, w=1
.....
.....
.....
.....
.....

z=-1, w=1
.....
.....
.....
.....
.....

z=0, w=1
.....
.....
.....
.....
.....

z=1, w=1
.....
.....
.....
.....
.....

z=2, w=1
.....
.....
.....
.....
.....

z=-2, w=2
.....
.....
..#..
.....
.....

z=-1, w=2
.....
.....
.....
.....
.....

z=0, w=2
###..
##.##
#...#
.#..#
.###.

z=1, w=2
.....
.....
.....
.....
.....

z=2, w=2
.....
.....
..#..
.....
.....
```
After the full six-cycle boot process completes, 848 cubes are left in the active state.

Starting with your given initial configuration, simulate six cycles in a 4-dimensional space. How many cubes are left in the active state after the sixth cycle?



In [123]:
class HyperCube(NamedTuple):
    w: int = 0 
    x: int = 0
    y: int = 0
    z: int = 0
    
    def __add__(self, other: Cube) -> Cube:
        return HyperCube(
            self.w + other.w,
            self.x + other.x,
            self.y + other.y,
            self.z + other.z,
        )
        
    @property
    def neighbors(self) -> Set[Cube]:
        offsets = (HyperCube(*wxyz) for wxyz in product([-1, 0, 1], repeat=4) if wxyz != (0, 0, 0, 0))
        return set(self + other for other in offsets)

In [124]:
g2 = Game.from_str(data, HyperCube)
g2.n_active

28

In [125]:
g2.step_n(6)

Initial active: 28
 After 1 cycles: 204
 After 2 cycles: 240
 After 3 cycles: 884
 After 4 cycles: 736
 After 5 cycles: 3412
 After 6 cycles: 2012
